# Import

In [1]:
import pandas as pd
import numpy as np
import string
import re
import random
import pickle
import nltk
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk import word_tokenize, pos_tag, regexp_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import WordNetLemmatizer 
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from sklearn.ensemble import RandomForestClassifier

In [2]:
train = pd.read_csv('train.csv')
holdout = pd.read_csv('test.csv')

In [3]:
test_labels_df = pd.read_csv("test_labels.csv")
holdout = holdout.merge(test_labels_df, on='id')
holdout.drop(holdout[holdout['toxic']==-1].index, inplace=True)

In [4]:
pickler = open("porter_stemmed_tokens.pickle", "rb")
porter_stems = pickle.load(pickler)
pickler.close()

In [5]:
pickler_test = open("porter_stemmed_tokens_test.pickle", "rb")
porter_stems_test = pickle.load(pickler_test)
pickler_test.close()

In [16]:
porter_stems[0]

"explan whi the edit made under my usernam hardcor metallica fan were revert ? they weren't vandal , just closur on some ga after i vote at new york doll fac . and pleas don't remov the templat from the talk page sinc i'm retir now . 89.205 . 38.27"

# Prepare X and Y

In [17]:
X_train = porter_stems
y_train = train.iloc[:,2:]

X_test = porter_stems_test
y_test = holdout.iloc[:,2:]

# Pipe Line

In [26]:
pipe_rf = make_pipeline(
    TfidfVectorizer(sublinear_tf=True, max_features=50000),
    OneVsRestClassifier(RandomForestClassifier(n_estimators=10))
)

In [24]:
cv_score = np.mean(cross_val_score(pipe_rf, X_train, y_train, cv=3, scoring='roc_auc'))

In [25]:
cv_score

0.9026193152690216

In [27]:
tfidfstring = 'tfidfvectorizer'
onevrstring = 'onevsrestclassifier'
param_grid_rf = {
#     tfidfstring+'__max_features': [25000,50000,100000],
#     tfidfstring+'__norm': ['l1','l2'],
#     tfidfstring+'__sublinear_tf': [True, False],
#     onevrstring+'__estimator__max_depth': [50,60],
    onevrstring+'__estimator__min_samples_split':[2,4]
             }

In [28]:
grid_rf = GridSearchCV(pipe_rf, param_grid_rf, cv=3, scoring='roc_auc')
grid_results_rf = grid_rf.fit(X_train, y_train)

In [13]:
grid_results_rf.best_params_

{'onevsrestclassifier__estimator__max_depth': 50}

In [14]:
grid_results_rf.best_score_

0.5002741644786188